# xml2yolo

In [ ]:
import xml.etree.ElementTree as ET
import glob
import os
import json

In [85]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [86]:
classes = []
input_dir = "C:\\Users\\hp\\Desktop\\BD2\\annotations"
output_dir = "C:\\Users\\hp\\Desktop\\BD2\\yolo_labels"
image_dir = "C:\\Users\\hp\\Desktop\\BD2\\images"

In [87]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [88]:
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue
    result = []
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    for obj in root.findall('object'):
        label = obj.find("name").text
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")
    if result:
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

In [ ]:
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

# draw

In [90]:
from PIL import Image, ImageDraw

In [91]:
def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]
def draw_image(img, bboxes):
    draw = ImageDraw.Draw(img)
    for bbox in bboxes:
        draw.rectangle(bbox, outline="red", width=2)
    img.show()

In [208]:
image_filename = f"C:\\Users\\hp\\Desktop\\BD2\\images\\boat1.png"
label_filename = f"C:\\Users\\hp\\Desktop\\BD2\\yolo_labels\\boat1.txt"
bboxes = []

In [209]:
img = Image.open(image_filename)
with open(label_filename, 'r', encoding='utf8') as f:
    for line in f:
        data = line.strip().split(' ')
        bbox = [float(x) for x in data[1:]]
        bboxes.append(yolo_to_xml_bbox(bbox, img.width, img.height))

In [210]:
draw_image(img, bboxes)

# split_datasets

In [218]:
import os
import shutil
import glob
import random

def copyfiles(fil, folder):
    # Fonction copyfiles() pour copier les fichiers vers le dossier spécifié
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    
    # Chemins d'origine et de destination pour les images
    src_image = os.path.join("C:\\Users\\hp\\Desktop\\BD2\\images", f"{filename}.png")
    dest_image = os.path.join(folder, "images", f"{filename}.png")
    shutil.copyfile(src_image, dest_image)
    
    # Chemins d'origine et de destination pour les étiquettes
    src_label = os.path.join("C:\\Users\\hp\\Desktop\\BD2\\yolo_labels", f"{filename}.txt")
    dest_label = os.path.join(folder, "yolo_labels", f"{filename}.txt")
    shutil.copyfile(src_label, dest_label)
    
    # Chemins d'origine et de destination pour les annotations
    src_annotation = os.path.join("C:\\Users\\hp\\Desktop\\BD2\\annotations", f"{filename}.xml")
    dest_annotation = os.path.join(folder, "annotations", f"{filename}.xml")
    shutil.copyfile(src_annotation, dest_annotation)

# Chemins des répertoires sources
image_dir = "C:\\Users\\hp\\Desktop\\BD2\\images"
label_dir = "C:\\Users\\hp\\Desktop\\BD2\\yolo_labels"
annotation_dir = "C:\\Users\\hp\\Desktop\\BD2\\annotations"

# Chemin du répertoire racine pour les dossiers train, test, valid
root_dir = "C:\\Users\\hp\\Desktop\\BD2"

# Proportions de division pour les dossiers train, test, valid
folders = {"train": 0.6, "test": 0.2, "valid": 0.2}

# Vérification de la somme des proportions
check_sum = sum(folders.values())
assert check_sum == 1.0, "La somme des proportions n'est pas égale à 1.0"

# Création des dossiers train, test, valid
for folder in folders:
    os.makedirs(os.path.join(root_dir, folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(root_dir, folder, "yolo_labels"), exist_ok=True)
    os.makedirs(os.path.join(root_dir, folder, "annotations"), exist_ok=True)

# Obtention de la liste des fichiers d'images
files = glob.glob(os.path.join(image_dir, '*.png'))
random.shuffle(files)

# Boucle pour la division des fichiers
lower_limit = 0
for folder in folders:
    limit = round(len(files) * folders[folder])
    for fil in files[lower_limit:lower_limit + limit]:
        copyfiles(fil, os.path.join(root_dir, folder))
    lower_limit += limit


# training

In [4]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [5]:
%cd yolov5

C:\Users\hp\Desktop\BD2\yolov5


In [6]:
%pip install -qr requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
import utils 

In [8]:
display= utils.notebook_init()

YOLOv5  v7.0-177-g89c3040 Python-3.10.9 torch-1.13.1 CPU


Setup complete  (8 CPUs, 15.8 GB RAM, 213.0/237.5 GB disk)


In [9]:
!python train.py --img 640 --batch 1 --epochs 3 --data /content/yolov5/data/custom.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/custom.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
From https://github.com/ultralytics/yolov5
   89c3040..a199480  master     -> origin/master
github:  YOLOv5 is out of date by 1 commit. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
Traceback (most recent call last):
  File "C:\Users\hp\Desktop\BD2\yolov5\train.py", line 642, in <module>
    main(opt)
  File "C:\Users\hp\Desktop\BD2\yolov5\train.py", line 506,

In [11]:
!python detect.py --source C:\Users\hp\Desktop\BD2\test\images\boat127.png

detect: weights=yolov5s.pt, source=C:\Users\hp\Desktop\BD2\test\images\boat127.png, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-177-g89c3040 Python-3.10.9 torch-1.13.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
image 1/1 C:\Users\hp\Desktop\BD2\test\images\boat127.png: 608x640 1 scissors, 291.2ms
Speed: 3.7ms pre-process, 291.2ms inference, 29.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp5


In [12]:
!pip install --upgrade yolov5


     -------------------------------------- 953.3/953.3 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 56.3/56.3 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 135.6/135.6 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 104.0/104.0 kB ? eta 0:00:00
     ------------------------------------- 236.8/236.8 kB 14.2 MB/s eta 0:00:00
     ---------------------------------------- 88.3/88.3 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 10.8/10.8 MB 11.9 MB/s eta 0:00:00
     ---------------------------------------- 79.8/79.8 kB ? eta 0:00:00
     ------------------------------------- 160.1/160.1 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 54.5/54.5 kB ? eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): star

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiohttp 3.8.3 requires charset-normalizer<3.0,>=2.0, but you have charset-normalizer 3.0.1 which is incompatible.
